In [ ]:
# import packages
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
from collections import defaultdict
import numpy as np

# plotting libraries
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import plotly.graph_objects as go

In [ ]:
base_url = "https://understat.com/league/EPL"

In [ ]:
res = requests.get(base_url)
matches = BeautifulSoup(res.content, "lxml")

In [ ]:
matches = matches.find_all("script")

In [ ]:
matches = matches[1].string

In [ ]:
start = matches.index("(") + 2
end = matches.index(")")

matches = matches[start:end]

In [ ]:
json_data = matches.encode("utf8").decode("unicode_escape")
json_data = json.loads(json_data[:-1])

In [ ]:
json_data[0]

In [ ]:
weekly_table = defaultdict(list)

for match in json_data:
    if not match["isResult"]:
        continue
    if match["goals"]["h"] > match["goals"]["a"]:
        pts_h = 3
        pts_a = 0
    elif match["goals"]["h"] < match["goals"]["a"]:
        pts_h = 0
        pts_a = 3
    else:
        pts_h = 1
        pts_a = 1
    
    weekly_table[match["h"]["title"]].append(pts_h)
    weekly_table[match["a"]["title"]].append(pts_a)

In [ ]:
cumulative_table = defaultdict(list)

for team, weeks in weekly_table.items():
    curr_table = []
    j = 0
    for wk in weeks:
        j += wk
        curr_table.append(j)
    cumulative_table[team] = curr_table

In [ ]:
team_colors = {
    "Arsenal": "#EF0107",
    "Aston Villa": "#95BFE5",
    "Brentford": "#E30613",
    "Brighton": "#0057B8",
    "Burnley": "#6C1D45",
    "Chelsea": "#034694",
    "Crystal Palace": "#1B458F",
    "Everton": "#003399",
    "Leeds": "#1D428A",
    "Leicester": "#003090",
    "Liverpool": "#C8102E",
    "Manchester City": "#6CABDD",
    "Manchester United": "#DA291C",
    "Newcastle United": "#241F20",
    "Norwich": "#00A650",
    "Southampton": "#D71920",
    "Tottenham": "#132257",
    "Watford": "#11210C",
    "West Ham": "#7A263A",
    "Wolverhampton Wanderers": "#231F20"
}

In [ ]:
league_order = [
    "Manchester City",
    "Liverpool",
    "Chelsea",
    "Tottenham",
    "Arsenal",
    "Manchester United",
    "West Ham",
    "Leicester",
    "Brighton",
    "Wolverhampton Wanderers",
    "Newcastle United",
    "Crystal Palace",
    "Brentford",
    "Aston Villa",
    "Southampton",
    "Everton",
    "Leeds",
    "Burnley",
    "Watford",
    "Norwich"
]

In [ ]:
plt.rcParams["font.family"] = "Larsseit"
plt.rcParams["font.size"] = 18

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111, frameon=False)

min_pts = cumulative_table[league_order[-1]][-1]
max_pts = cumulative_table[league_order[0]][-1]

i = 20
for team in league_order:
    cum_pts = cumulative_table[team]
    ax.plot([i for i in range(1, len(cum_pts) + 1)], cum_pts, lw=10, color=team_colors[team], alpha=0.2)
    ax.plot([i for i in range(1, len(cum_pts) + 1)], cum_pts, lw=2, marker="o", color=team_colors[team])
    ax.text(38, min_pts + i * (max_pts - min_pts) / 20, team, ha="left", va="center")
    i -= 1
    
# axis limits    
plt.show()

In [ ]:
plt.rcParams["font.family"] = "Fantasque Sans Mono"
plt.rcParams["font.size"] = 18

fig = plt.figure(figsize=(16, 16), facecolor="#F9F9EB")
ax = fig.add_subplot(111, frameon=False)

min_pts = cumulative_table[league_order[-1]][-1]
max_pts = cumulative_table[league_order[0]][-1]

# champions league
rect_ucl = Rectangle((0.7, 20.25), 11.5, 4.5, color="forestgreen", alpha=0.1)
ax.add_artist(rect_ucl)
ax.text(13, 22.5, "Champions League", ha="left", va="center", fontweight="bold")

# europa league
rect_uel = Rectangle((0.7, 17.25), 12.5, 2.5, color="slateblue", alpha=0.1)
ax.add_artist(rect_uel)
ax.text(14, 18.5, "Europa League", ha="left", va="center", fontweight="bold")

# europa conference league
rect_uecl = Rectangle((0.7, 15.25), 8.5, 1.5, color="maroon", alpha=0.1)
ax.add_artist(rect_uecl)
ax.text(10, 16, "Europa Conference League", ha="left", va="center", fontweight="bold")

# relegation zone
rect_rel = Rectangle((0.7, 0.25), 8, 3.5, color="red", alpha=0.1)
ax.add_artist(rect_rel)
ax.text(9.5, 2, "Relegation Zone", ha="left", va="center", fontweight="bold")

r = 1
for i, team in enumerate(league_order[::-1]):
    if i in [3, 13, 14, 16]:
        r += 1
    pts = cumulative_table[team]
    theta = np.linspace(np.pi / 2, (pts[-1] / max_pts) * (270 * np.pi / 180) + np.pi / 2, 1000)
    ax.plot(r * np.cos(theta), r * np.sin(theta), lw=10, color=team_colors[team], alpha=0.2)
    ax.plot(r * np.cos(theta), r * np.sin(theta), lw=4, color=team_colors[team])
    ax.text(r * np.cos(theta[0]) + 1, r * np.sin(theta[0]), f"{team} - {pts[-1]} pts", ha="left", va="center", size=15, color=team_colors[team])
    r += 1
    
# tick marks
ax.set(xticks=[], yticks=[])

# add title
ax.text(0, 1, "English Premier League Table", transform=ax.transAxes, ha="left", va="top", size=24, fontweight="bold")
ax.text(0, 0.97, "2021-22 SEASON", transform=ax.transAxes, ha="left", va="top", size=22, color="gray")

# add reference
ax.text(
    0,
    0,
    "Source: Understat",
    transform=ax.transAxes,
    ha="left",
    va="bottom",
    size=15
)
ax.text(
    1,
    0,
    "@naveenv_92",
    transform=ax.transAxes,
    ha="right",
    va="bottom",
    size=15
)

plt.savefig("epl-table.png", dpi=500, bbox_inches="tight")
plt.show()